In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from arch import arch_model
import statsmodels.api as sm
from scipy.stats import norm, t
import seaborn as sns

In [ ]:
kospi = pd.read_csv("^kospi_d.csv")
ndx = pd.read_csv("^ndx_d.csv")
kospi["Return"] = 100 * (kospi["Close"] - kospi["Close"].shift(1)) / kospi["Close"].shift(1)
k = kospi.dropna()
ndx["Return"] = 100 * (ndx["Close"] - ndx["Close"].shift(1)) / ndx["Close"].shift(1)
n = ndx.dropna()

In [ ]:
n['Date'] = pd.to_datetime(n['Date'])
k['Date'] = pd.to_datetime(k['Date'])

In [ ]:
# Load and prepare dataset function
def load_and_prepare_data(filename):
    data = pd.read_csv(filename, parse_dates=["Date"], index_col="Date")
    data["Return"] = 100 * (data["Close"] - data["Close"].shift(1)) / data["Close"].shift(1)
    return data.dropna()

# Load datasets
ndx_data = load_and_prepare_data("^ndx_d.csv")
kospi_data = load_and_prepare_data("^kospi_d.csv")

In [ ]:
# Align datasets to start on the same date
# common_start_date = max(ndx_data.index.min(), kospi_data.index.min())
# ndx_data, kospi_data = ndx_data.loc[common_start_date:], kospi_data.loc[common_start_date:]

# Q1 & 2

In [ ]:
# Compute statistics function
def compute_stats(data):
    returns = data["Return"]
    return {
        "Start Date": returns.index.min(),
        "End Date": returns.index.max(),
        "Observations": len(returns),
        "Mean Return": returns.mean(),
        "Variance": returns.var(),
        "Standard Deviation": returns.std(),
        "Minimum Return": returns.min(),
        "Maximum Return": returns.max(),
        "Median Return": returns.median(),
        "Skewness": stats.skew(returns),
        "Kurtosis": stats.kurtosis(returns),
    }

# Print statistics function
def print_stats(name, stats_dict):
    print(f"""
{name}
- Observations: {stats_dict["Observations"]:,}
- Start Date: {stats_dict["Start Date"]:}
- End Date: {stats_dict["End Date"]:}
- Mean daily return: {stats_dict["Mean Return"]:.3f}
- Variance: {stats_dict["Variance"]:.3f}
- Standard deviation: {stats_dict["Standard Deviation"]:.5f}
- Minimum return: {stats_dict["Minimum Return"]:.3f}%
- Maximum return: {stats_dict["Maximum Return"]:.3f}%
- Median return: {stats_dict["Median Return"]:.3f}%
- Skewness: {stats_dict["Skewness"]:.3f}
- Kurtosis: {stats_dict["Kurtosis"]:.3f}
""")

# Compute and print statistics
ndx_stats, kospi_stats = compute_stats(ndx_data), compute_stats(kospi_data)
print_stats("NASDAQ-100 (^NDX)", ndx_stats)
print_stats("KOSPI (^KOSPI)", kospi_stats)

In [ ]:
# Compute mean and standard deviation for both indices
ndx_mu, ndx_sigma = ndx_data["Return"].mean(), ndx_data["Return"].std()
kospi_mu, kospi_sigma = kospi_data["Return"].mean(), kospi_data["Return"].std()

# Generate x-values for normal distributions
x_values = np.linspace(min(ndx_data["Return"].min(), kospi_data["Return"].min()), 
                       max(ndx_data["Return"].max(), kospi_data["Return"].max()), 500)

# Compute normal density curves
ndx_norm = norm.pdf(x_values, ndx_mu, ndx_sigma)
kospi_norm = norm.pdf(x_values, kospi_mu, kospi_sigma)

# Plot KDE for NDX and KOSPI
plt.figure(figsize=(10, 5))
sns.kdeplot(ndx_data["Return"], color="blue", label="NDX KDE", linewidth=3)
sns.kdeplot(kospi_data["Return"], color="red", label="KOSPI KDE", linewidth=3)

# Plot fitted normal distributions
plt.plot(x_values, ndx_norm, color="blue", linestyle="dashed", label="NDX Normal", linewidth=1)
plt.plot(x_values, kospi_norm, color="red", linestyle="dashed", label="KOSPI Normal", linewidth=1)

# Labels and legend
plt.xlabel("Daily Returns")
plt.ylabel("Density")
plt.legend()
# plt.title("NDX vs KOSPI Return Density with Normal Overlay")

# Show plot
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot NASDAQ-100 returns
ax.plot(ndx_data.index, ndx_data["Return"], color="blue", alpha=0.5, label="NDX")

# Plot KOSPI returns
ax.plot(kospi_data.index, kospi_data["Return"], color="red", alpha=0.5, label="KOSPI")

# Add horizontal line at 0%
ax.axhline(0, linestyle="--", color="black", linewidth=1)

# Labels and title
# ax.set_title("NASDAQ-100 vs. KOSPI Daily Returns (%)")
ax.set_xlabel("Date")
ax.set_ylabel("Return (%)")

# Add legend
ax.legend()

# Improve layout
plt.tight_layout()

# Show plot
plt.show()

# Q3

In [ ]:
# Set parameters for VaR
var_levels = [0.01, 0.05, 0.10]
update_freq = 1  # Daily updates

# Function to compute FHS VaR with a rolling 10-year window
def filtered_historical_simulation(returns, update_freq, var_levels):
    start_date = returns.index.min() + pd.DateOffset(years=10)
    dates = returns.loc[start_date:].index  
    var_results = {alpha: [] for alpha in var_levels}

    for date in dates[::update_freq]:
        train_data = returns.loc[date - pd.DateOffset(years=10):date]  # 10-year rolling window

        # Fit GARCH(1,1)
        garch_model = arch_model(train_data, vol='Garch', p=1, q=1, mean='Constant')
        garch_fit = garch_model.fit(disp="off")

        # Compute standardized residuals & forecast next period's volatility
        residuals = garch_fit.std_resid
        forecast = garch_fit.forecast(horizon=1, reindex=False).variance.iloc[-1, 0]
        forecast_vol = np.sqrt(forecast)
        rolling_mean = garch_fit.params['mu']

        # Compute VaR at different levels
        for alpha in var_levels:
            var_results[alpha].append((date, -rolling_mean - forecast_vol * residuals.quantile(alpha, interpolation='higher')))

    return var_results

# Compute VaR forecasts
ndx_var = filtered_historical_simulation(ndx_data["Return"], update_freq, var_levels)
kospi_var = filtered_historical_simulation(kospi_data["Return"], update_freq, var_levels)

# Convert results to DataFrame
def var_results_to_df(var_results, var_levels):
    return pd.DataFrame({f"VaR {int((1 - alpha) * 100)}%": [v[1] for v in var_results[alpha]] for alpha in var_levels},
                        index=[v[0] for v in var_results[var_levels[0]]])

ndx_var_df = var_results_to_df(ndx_var, var_levels)
kospi_var_df = var_results_to_df(kospi_var, var_levels)

# Export results
def save_var_results(df, filename):
    filepath = filename if filename not in locals() else f"new_{filename}"
    df.to_csv(filepath, index_label="Date")
    print(f"VaR results saved to {filepath}")

save_var_results(ndx_var_df, "ndx_var_results.csv")
save_var_results(kospi_var_df, "kospi_var_results.csv")

# Plot VaR estimates
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ndx_var_df.plot(ax=axes[0], title="Filtered Historical Simulation VaR for NDX")
axes[0].set_ylabel("VaR (%)")

kospi_var_df.plot(ax=axes[1], title="Filtered Historical Simulation VaR for KOSPI")
axes[1].set_ylabel("VaR (%)")

plt.xlabel("Date")
plt.show()


In [ ]:
# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(n['Date'], n['Close'], label="NDX Closing Price", color="blue", alpha=0.7)
plt.plot(k['Date'], k['Close'], label="KOSPI Closing Price", color="red", alpha=0.7)

# Formatting
plt.xlabel("Date")
plt.ylabel("Returns")
# plt.title("NDX vs KOSPI Returns Over Time")
plt.legend()
plt.grid(True)

# Show plot
plt.show()

# Q4

## Generalised Error

In [ ]:
# Function to compute HITs (VaR violations) and export results
def compute_and_export_hits(index_name, returns, var_series_dict, output_filename):
    hit_data = {}

    # Ensure all data aligns to the same timestamps
    common_index = returns.index  # Start with the returns index
    for model_name, var_series in var_series_dict.items():
        common_index = common_index.intersection(var_series.index)  # Keep only common dates

    # Align all series to common dates
    returns = returns.loc[common_index]

    for model_name, var_series in var_series_dict.items():
        var_series = var_series.loc[common_index]  # Align VaR series
        var_actual = -var_series  # Convert to negative values for correct interpretation
        hit_series = (returns < var_actual).astype(float)  # Compute HITs

        # Store only HITs
        hit_data[f"HIT_{model_name}"] = hit_series

    # Convert to DataFrame (ONLY HIT columns)
    hit_df = pd.DataFrame(hit_data).dropna()

    # Save to CSV (ONLY HITs)
    hit_df.to_csv(output_filename, index_label="Date")

    # Print HIT rates for each model
    for model_name in var_series_dict.keys():
        print(f"{index_name} HIT rate ({model_name}): {hit_df[f'HIT_{model_name}'].mean():.4f}")

    # Compute and print correlation of HITs between models (if multiple models exist)
    if len(hit_data) > 1:
        hit_correlation = hit_df.corr()
        print(f"Correlation of HITs between models:\n{hit_correlation}")

    return hit_df

# Function to plot VaR violations (HITs)
def plot_hits(hit_dfs, title):
    plt.figure(figsize=(16, 6))

    # Fixed y-axis offsets to maintain clear visibility
    offsets = [1.0, 0.97]

    # Extract the two datasets
    (index_name_1, hit_df_1), (index_name_2, hit_df_2) = hit_dfs.items()
    
    # Get the first HIT column from each dataset
    col_1 = [c for c in hit_df_1.columns if "HIT_" in c][0]
    col_2 = [c for c in hit_df_2.columns if "HIT_" in c][0]

    # Prepare the plots with fixed offsets
    hit_plot_1 = hit_df_1[col_1].replace(0.0, np.nan).apply(lambda x: offsets[0] if x == 1 else np.nan)
    hit_plot_2 = hit_df_2[col_2].replace(0.0, np.nan).apply(lambda x: offsets[1] if x == 1 else np.nan)

    # Scatter plot with fixed markers
    plt.scatter(hit_plot_1.index, hit_plot_1, marker=".", s=15, label=f"{index_name_1} - {col_1.replace('HIT_', '')}")
    plt.scatter(hit_plot_2.index, hit_plot_2, marker=".", s=15, label=f"{index_name_2} - {col_2.replace('HIT_', '')}")

    plt.ylim(0.95, 1.02)  # Maintain spacing similar to the original
    plt.yticks([])  # Remove unnecessary y-axis ticks
    plt.ylabel("VaR Violations")
    plt.xlabel("Date")
    plt.legend(frameon=False)
    plt.title(title)
    plt.show()

In [ ]:
# Compute HITs for FHS VaR (Filtered Historical Simulation)
ndx_hits_fhs = compute_and_export_hits("NDX (FHS)", ndx_data["Return"], {"VaR 95%": ndx_var_df["VaR 95%"]}, "ndx_combined.csv")
kospi_hits_fhs = compute_and_export_hits("KOSPI (FHS)", kospi_data["Return"], {"VaR 95%": kospi_var_df["VaR 95%"]}, "kospi_combined.csv")

# Plot HITs for FHS VaR
# "HIT Plot (VaR Violations) for NDX and KOSPI - FHS"
plot_hits({"NDX": ndx_hits_fhs, "KOSPI": kospi_hits_fhs}, title="")


In [ ]:
# Function to compute likelihood ratio test for each column in hits_data
def likelihood_ratio_test(index_name, hits_data, expected_rate=0.05):
    for col in hits_data:
        hit = hits_data[col]
        phat = hit.mean()  # Observed hit rate for this column
        llf = stats.bernoulli(phat).logpmf(hit).sum()  # Log-likelihood of observed rate
        llf0 = stats.bernoulli(expected_rate).logpmf(hit).sum()  # Log-likelihood under expected rate
        lr = 2 * (llf - llf0)  # Likelihood ratio test statistic
        pval = 1 - stats.chi2(1).cdf(lr)  # P-value

        print(f"{index_name} LR: {lr:.4f} P-value: {pval:.4f}")

# Apply to datasets
likelihood_ratio_test("NDX", ndx_hits_fhs)
likelihood_ratio_test("KOSPI", kospi_hits_fhs)

In [ ]:
# Perform Christoffersen's Independence Test for both indices
def christoffersen_independence_test(index_name, hits_data):
    hits_data_new = hits_data.squeeze()
    hit_t = hits_data_new.shift(1)  # Previous day's HIT
    hit_tp1 = hits_data_new  # Current day's HIT

    # Count transitions
    n00 = ((1 - hit_t) * (1 - hit_tp1)).sum()
    n10 = (hit_t * (1 - hit_tp1)).sum()
    n01 = ((1 - hit_t) * hit_tp1).sum()
    n11 = (hit_t * hit_tp1).sum()

    # Compute transition probabilities
    p00_hat = n00 / (n00 + n01)
    p11_hat = n11 / (n11 + n10)
    p00 = p00_hat
    p11 = p11_hat
    # Compute log-likelihood for observed transition probabilities
    llf = (
        n00 * np.log(p00)
        + n10 * np.log(1 - p00)
        + n11 * np.log(p11)
        + n10 * np.log(1 - p11)
    )


    # Compute log-likelihood under independence assumption
    p11 = 0.05
    p00 = 1 - p11
    llf0 = (
        n00 * np.log(p00)
        + n10 * np.log(1 - p00)
        + n11 * np.log(p11)
        + n10 * np.log(1 - p11)
    )

    # Compute likelihood ratio test statistic
    lr = 2 * (llf - llf0)
    pval = 1 - stats.chi2(2).cdf(lr)

    print(f"Christoffersen's Test, {index_name} LR: {lr:.4f} P-value: {pval:.4f}")

# Apply Christoffersen's test to NDX and KOSPI
for index_name, hits_data in zip(["NDX", "KOSPI"], [ndx_hits_fhs, kospi_hits_fhs]):
    christoffersen_independence_test(index_name, hits_data)

In [ ]:
# Function to perform OLS regression and Wald test
def ols_regression_with_wald_test(index_name, hits_data, var_df, var_col, expected_violation_rate=0.05):

    hit = hits_data - expected_violation_rate  # Adjust violations by subtracting expected rate

    # Create lagged HIT variables (5 lags)
    lags = pd.concat([hit.shift(i + 1) for i in range(5)], axis=1)

    # Select corresponding VaR column
    var = var_df[var_col]

    # Combine data for regression
    data = pd.concat([hit, var, lags], axis=1).dropna()
    y = data.iloc[:, 0]  # Dependent variable (adjusted HIT)
    x = sm.add_constant(data.iloc[:, 1:])  # Add constant term
    x.columns = ["const", "VaR"] + [f"hit_L_{i}" for i in range(1, 6)]  # Rename columns

    # Run OLS regression
    res = sm.OLS(y, x).fit()

    # Wald test for joint significance of all coefficients (except constant)
    r = np.eye(len(x.columns))  
    joint = res.wald_test(r, scalar=True)
    
    return {
        "summary": res.summary(),
        "stat": f"Stat: {joint.statistic:.4f}, P-value: {joint.pvalue:.4f}"
    }

# Store results for FHS VaR HITs
results_fhs = {
    "NDX": ols_regression_with_wald_test("NDX", ndx_hits_fhs, ndx_var_df, "VaR 95%"),
    "KOSPI": ols_regression_with_wald_test("KOSPI", kospi_hits_fhs, kospi_var_df, "VaR 95%"),
}

# Print results for FHS
for index_name in ["NDX", "KOSPI"]:
    print(f"OLS Regression Summary for {index_name} (FHS):")
    print(results_fhs[index_name]["summary"])
    print(f"Wald Test {index_name} (FHS): {results_fhs[index_name]['stat']}\n")


In [ ]:
# Function to perform Logit and Probit regression
def logit_probit_regression(index_name, hits_data, var_df, var_col):

    hit = hits_data  # HIT variable (already 0 or 1)

    # Create lagged HIT variables (5 lags)
    lags = pd.concat([hit.shift(i + 1) for i in range(5)], axis=1)

    # Select corresponding VaR column
    var = var_df[var_col]

    # Combine data for regression
    data = pd.concat([hit, var, lags], axis=1).dropna()
    y = data.iloc[:, 0]  # Binary target variable (HIT)
    x = sm.add_constant(data.iloc[:, 1:])  # Add intercept
    x.columns = ["const", "VaR"] + [f"hit_L_{i}" for i in range(1, 6)]  # Rename columns

    # Run Logit and Probit models
    logit_model = sm.Logit(y, x).fit(disp=0)
    probit_model = sm.Probit(y, x).fit(disp=0)

    # Wald test for joint significance of all coefficients (except constant)
    r = np.eye(len(x.columns))  
    joint_logit = logit_model.wald_test(r, scalar=True)
    joint_probit = probit_model.wald_test(r, scalar=True)

    return {
        "logit_summary": logit_model.summary(),
        "logit_stat": f"Stat: {joint_logit.statistic:.4f}, P-value: {joint_logit.pvalue:.4f}",
        "probit_summary": probit_model.summary(),
        "probit_stat": f"Stat: {joint_probit.statistic:.4f}, P-value: {joint_probit.pvalue:.4f}"
    }

# Store results for FHS VaR HITs
results_fhs_logit_probit = {
    "NDX": logit_probit_regression("NDX", ndx_hits_fhs, ndx_var_df, "VaR 95%"),
    "KOSPI": logit_probit_regression("KOSPI", kospi_hits_fhs, kospi_var_df, "VaR 95%"),
}

# Print results for FHS
for index_name in ["NDX", "KOSPI"]:
    print(f"Logit Regression Summary for {index_name} (FHS):")
    print(results_fhs_logit_probit[index_name]["logit_summary"])
    print(f"\nProbit Regression Summary for {index_name} (FHS):")
    print(results_fhs_logit_probit[index_name]["probit_summary"])


# Q5

In [ ]:
# Set decay factor for EWMA (RiskMetrics)
lambda_ewma = 0.94  # Daily data

# Function to compute RiskMetrics VaR using explicit summation
def riskmetrics_var_explicit(data, lambda_ewma, var_levels):
    ewma_var = np.zeros(len(data))  # Store variance estimates
    
    # Compute EWMA variance using explicit summation
    for t in range(len(data)):
        past_returns = data.iloc[:t+1]  # Take all past data up to time t
        weights = (1 - lambda_ewma) * lambda_ewma ** np.arange(len(past_returns))[::-1]  # Compute correct decay weights
        ewma_var[t] = np.sum(weights * past_returns**2)  # Apply formula

    ewma_vol = np.sqrt(ewma_var)  # Convert variance to standard deviation

    # Compute VaR for different confidence levels using normal quantiles
    var_results = {alpha: -ewma_vol * stats.norm.ppf(alpha) for alpha in var_levels}  # % VaR

    return pd.DataFrame(var_results, index=data.index)

# Compute RiskMetrics VaR for NDX and KOSPI using explicit summation
ndx_var_riskmetrics_explicit = riskmetrics_var_explicit(ndx_data["Return"].dropna(), lambda_ewma, [0.01, 0.05, 0.10])
kospi_var_riskmetrics_explicit = riskmetrics_var_explicit(kospi_data["Return"].dropna(), lambda_ewma, [0.01, 0.05, 0.10])

# Find the first date where FHS VaR is available
first_ndx_var_date = ndx_var_df.index.min()
first_kospi_var_date = kospi_var_df.index.min()

# Trim RiskMetrics VaR data to start from the same date as FHS VaR
ndx_var_riskmetrics_explicit = ndx_var_riskmetrics_explicit.loc[first_ndx_var_date:]
kospi_var_riskmetrics_explicit = kospi_var_riskmetrics_explicit.loc[first_kospi_var_date:]

# Plot RiskMetrics VaR
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# NDX VaR
ndx_var_riskmetrics_explicit.plot(ax=axes[0], title="RiskMetrics VaR (EWMA) for NDX")
axes[0].set_ylabel("VaR (%)")

# KOSPI VaR
kospi_var_riskmetrics_explicit.plot(ax=axes[1], title="RiskMetrics VaR (EWMA) for KOSPI")
axes[1].set_ylabel("VaR (%)")

plt.xlabel("Date")
plt.show()


In [ ]:
# Compare 5% alpha results of FHS VaR and RiskMetrics VaR
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot 95% VaR for NDX
axes[0].plot(ndx_var_df.index, ndx_var_df["VaR 95%"], label="FHS VaR 95% (NDX)", linestyle="-", alpha=0.7)
axes[0].plot(ndx_var_riskmetrics_explicit.index, ndx_var_riskmetrics_explicit[0.05], label="RiskMetrics VaR 5% (NDX)", linestyle="-", alpha=0.7)
axes[0].set_title("Comparison of 95% FHS VaR and 5% RiskMetrics VaR for NDX")
axes[0].set_ylabel("VaR (%)")
axes[0].legend()

# Plot 95% VaR for KOSPI
axes[1].plot(kospi_var_df.index, kospi_var_df["VaR 95%"], label="FHS VaR 95% (KOSPI)", linestyle="-", alpha=0.7)
axes[1].plot(kospi_var_riskmetrics_explicit.index, kospi_var_riskmetrics_explicit[0.05], label="RiskMetrics VaR 5% (KOSPI)", linestyle="-", alpha=0.7)
axes[1].set_title("Comparison of 95% FHS VaR and 5% RiskMetrics VaR for KOSPI")
axes[1].set_ylabel("VaR (%)")
axes[1].set_xlabel("Date")
axes[1].legend()

plt.show()


In [ ]:
# Compute HITs for RiskMetrics VaR (EWMA model)
ndx_hits_riskmetrics = compute_and_export_hits("NDX (RiskMetrics)", ndx_data["Return"], {"VaR 95%": ndx_var_riskmetrics_explicit[0.05]}, "ndx_combined2.csv")
kospi_hits_riskmetrics = compute_and_export_hits("KOSPI (RiskMetrics)", kospi_data["Return"], {"VaR 95%": kospi_var_riskmetrics_explicit[0.05]}, "kospi_combined2.csv")

# Plot HITs for RiskMetrics VaR
plot_hits({"NDX": ndx_hits_riskmetrics, "KOSPI": kospi_hits_riskmetrics}, "HITs (VaR Violations) for NDX and KOSPI - RiskMetrics")

In [ ]:
# Apply likelihood ratio test for RiskMetrics VaR HITs
for index_name, hits_data in zip(["NDX", "KOSPI"], [ndx_hits_riskmetrics, kospi_hits_riskmetrics]):
    likelihood_ratio_test(index_name, hits_data)

In [ ]:
# Apply Christoffersen's Independence Test for RiskMetrics VaR HITs
for index_name, hits_data in zip(["NDX", "KOSPI"], [ndx_hits_riskmetrics, kospi_hits_riskmetrics]):
    christoffersen_independence_test(index_name, hits_data)

In [ ]:
# Store results for RiskMetrics VaR HITs
results_riskmetrics = {
    "NDX": ols_regression_with_wald_test("NDX", ndx_hits_riskmetrics, ndx_var_riskmetrics_explicit, var_col=0.05),
    "KOSPI": ols_regression_with_wald_test("KOSPI", kospi_hits_riskmetrics, kospi_var_riskmetrics_explicit, var_col=0.05),
}

# Print results for RiskMetrics
for index_name in ["NDX", "KOSPI"]:
    print(f"OLS Regression Summary for {index_name} (RiskMetrics):")
    print(results_riskmetrics[index_name]["summary"])
    print(f"Wald Test {index_name} (RiskMetrics): {results_riskmetrics[index_name]['stat']}\n")


In [ ]:
# Store results for RiskMetrics VaR HITs
results_riskmetrics_logit_probit = {
    "NDX": logit_probit_regression("NDX", ndx_hits_riskmetrics, ndx_var_riskmetrics_explicit, var_col=0.05),
    "KOSPI": logit_probit_regression("KOSPI", kospi_hits_riskmetrics, kospi_var_riskmetrics_explicit, var_col=0.05),
}

# Print results for RiskMetrics
for index_name in ["NDX", "KOSPI"]:
    print(f"Logit Regression Summary for {index_name} (RiskMetrics):")
    print(results_riskmetrics_logit_probit[index_name]["logit_summary"])
    print(f"\nProbit Regression Summary for {index_name} (RiskMetrics):")
    print(results_riskmetrics_logit_probit[index_name]["probit_summary"])

In [ ]:
def tick_loss(returns, var_estimates, alpha=0.05):
    returns = returns.reindex(var_estimates.index)
    indicator = (returns < var_estimates).astype(int)  # Corrected logic
    return alpha * (returns - var_estimates) * (1 - indicator) + (1 - alpha) * (var_estimates - returns) * indicator

def diebold_mariano_test(index_name, returns, var_model_1, var_model_2, alpha=0.05):
    """ Diebold-Mariano test using OLS with HAC standard errors """

    # Convert VaRs to negative values (since VaR represents expected losses)
    var_model_1_actual = -var_model_1
    var_model_2_actual = -var_model_2

    # Compute tick losses
    loss_model_1 = tick_loss(returns, var_model_1_actual, alpha)
    loss_model_2 = tick_loss(returns, var_model_2_actual, alpha)

    # Compute loss differential
    d_t = loss_model_1 - loss_model_2
    d_t = d_t.dropna()

    # Define OLS regression model (d_t ~ constant)
    X = np.ones_like(d_t)  # Constant term for regression
    ols_model = sm.OLS(d_t, X).fit(cov_type="HAC", cov_kwds={"maxlags": int(1.2 * len(d_t) ** (1 / 3)), "use_correction": False})  # HAC standard errors

    # Extract key statistics
    dm_stat = ols_model.params.iloc[0] / ols_model.bse.iloc[0]
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))  # Two-tailed p-value

    print(f"DM Statistics for {index_name}: {dm_stat}, P-value: {p_value}")

In [ ]:
# Align daily returns with VaR data
ndx_returns = ndx_data["Return"].loc[ndx_var_riskmetrics_explicit.index]
kospi_returns = kospi_data["Return"].loc[kospi_var_riskmetrics_explicit.index]

# Align FHS VaR with RiskMetrics VaR for comparison
ndx_var_fhs_series = ndx_var_df["VaR 95%"].loc[ndx_var_riskmetrics_explicit.index]
kospi_var_fhs_series = kospi_var_df["VaR 95%"].loc[kospi_var_riskmetrics_explicit.index]

# Run Diebold-Mariano Test for NDX and KOSPI
diebold_mariano_test("NDX", ndx_returns, ndx_var_fhs_series, ndx_var_riskmetrics_explicit[0.05])
diebold_mariano_test("KOSPI", kospi_returns, kospi_var_fhs_series, kospi_var_riskmetrics_explicit[0.05])

# Q6

In [ ]:
# Load and preprocess KOSPI data
kospi = pd.read_csv("^kospi_d.csv", parse_dates=["Date"], index_col="Date")
kospi = kospi.sort_index()

# Resample to get Thursday closing prices (biweekly non-overlapping returns)
thursday_prices = kospi.resample("W-THU").last()
non_overlapping_thursdays = thursday_prices.iloc[::2]

# Compute non-overlapping returns
returns = (non_overlapping_thursdays["Close"] - non_overlapping_thursdays["Close"].shift(1)) / non_overlapping_thursdays["Close"].shift(1) * 100
returns = returns.dropna()

# Define initial 10-year rolling window
start_date = returns.index.min()
end_date = start_date + pd.DateOffset(years=10)
first_10_years_returns = returns.loc[start_date:end_date]

# Fit initial GARCH(1,1) model with a constant mean
garch_model = arch_model(first_10_years_returns, vol="Garch", p=1, q=1, mean="Constant")
garch_fit = garch_model.fit(disp="off")

# Forecast volatility for the next 2-week period
forecast = garch_fit.forecast(horizon=1)
volatility_forecast = np.sqrt(forecast.variance.iloc[-1].mean())

# Estimated mean return
estimated_mean = garch_fit.params["mu"]

# Compute 5% VaR assuming normal distribution
z_score = norm.ppf(0.05)
VaR_5_percent_norm = -estimated_mean - z_score * volatility_forecast

# Compute 5% VaR assuming Student's t-distribution
params = t.fit(first_10_years_returns)
degrees_of_freedom = params[0]
t_quantile = t.ppf(0.05, df=degrees_of_freedom)
VaR_5_percent_t = -estimated_mean - t_quantile * volatility_forecast

# Rolling window size
window_size = len(first_10_years_returns)

# Initialize lists to store VaR values
var_5_percent_norm_list, var_5_percent_t_list = [], []

# Rolling window estimation
for i in range(len(returns) - window_size):
    rolling_sample = returns.iloc[i : i + window_size]

    # Fit GARCH(1,1) model
    garch_model = arch_model(rolling_sample, vol="Garch", p=1, q=1, mean="Constant")
    garch_fit = garch_model.fit(disp="off")

    # Forecast volatility
    forecast = garch_fit.forecast(horizon=1)
    volatility_forecast = np.sqrt(forecast.variance.iloc[-1].mean())

    # Estimated mean return
    estimated_mean = garch_fit.params["mu"]

    # Compute VaR for both distributions
    VaR_5_p_norm = -estimated_mean - z_score * volatility_forecast
    VaR_5_p_t = -estimated_mean - t_quantile * volatility_forecast

    var_5_percent_norm_list.append(VaR_5_p_norm)
    var_5_percent_t_list.append(VaR_5_p_t)

# Convert the VaR lists to Series with timestamps
var_norm_series = pd.Series(var_5_percent_norm_list, index=returns.index[window_size:])
var_t_series = pd.Series(var_5_percent_t_list, index=returns.index[window_size:])

# Plot the two VaR series
plt.figure(figsize=(12, 8))
plt.plot(var_norm_series, label="VaR (Normal Dist.)", linestyle="-", color="blue", alpha=0.7)
plt.plot(var_t_series, label="VaR (Student's t Dist.)", linestyle="-", color="red", alpha=0.7)

# Add titles and labels
# plt.title("Rolling 5% 2-Week VaR (Normal vs. Student's t)", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Value-at-Risk (%)", fontsize=12)
plt.legend()
plt.grid(True, linestyle="-", alpha=0.5)

# Show the plot
plt.show()

var_t_series.to_csv("var_t_series.csv", index_label="Date")
var_norm_series.to_csv("var_normal_series.csv", index_label="Date")

print("VaR results saved as var_t_series.csv and var_normal_series.csv")

In [ ]:
# Ensure all datasets have matching timestamps
returns = returns.loc[var_norm_series.index]  # Align returns with the VaR series
var_t_series = var_t_series.loc[var_norm_series.index]

# Run Diebold-Mariano test for Normal VaR vs. t-distributed VaR
diebold_mariano_test("KOSPI", returns, var_t_series, var_norm_series, alpha=0.05)

# Q7 (a)

In [ ]:
# Load dataset and preprocess biweekly returns
kospi_data_new = pd.read_csv("^kospi_d.csv", parse_dates=["Date"], index_col="Date").sort_index()

# Resample to get Thursday closing prices (biweekly)
thursday_prices = kospi_data_new.resample("W-THU").last()
biweekly_prices = thursday_prices.iloc[::2]

# Compute biweekly returns and scale by 100
biweekly_returns = (biweekly_prices["Close"] - biweekly_prices["Close"].shift(1)) / biweekly_prices["Close"].shift(1) * 100
biweekly_returns = biweekly_returns.dropna()  # Remove NaNs from the first row

# Compute FHS VaR for 5% level using biweekly returns
kospi_var_fhs_biweekly = filtered_historical_simulation(biweekly_returns, update_freq=1, var_levels=[0.05])


# Convert dictionary to DataFrame
kospi_var_fhs_biweekly_df = pd.DataFrame(kospi_var_fhs_biweekly[0.05], columns=["Date", "VaR"]).set_index("Date")

# Plot the FHS VaR estimates
plt.figure(figsize=(12, 8))
kospi_var_fhs_biweekly_df["VaR"].plot(title="Filtered Historical Simulation 5% 2-Week VaR for KOSPI", legend=True)
plt.ylabel("VaR (%)")
plt.xlabel("Date")
plt.grid()
plt.show()

In [ ]:
# Ensure all series have the same date index range
vaR_combined = pd.concat([var_norm_series, var_t_series, kospi_var_fhs_biweekly_df["VaR"]], axis=1)
vaR_combined.columns = ["VaR (Normal Dist.)", "VaR (Student's t Dist.)", "VaR (FHS)"]
vaR_combined.dropna(inplace=True)  # Ensure aligned series

# Plot the three VaR estimates
plt.figure(figsize=(12, 8))
plt.plot(vaR_combined.index, vaR_combined["VaR (Normal Dist.)"], label="VaR (Normal Dist.)", linestyle="-", color="blue", alpha=0.7)
plt.plot(vaR_combined.index, vaR_combined["VaR (Student's t Dist.)"], label="VaR (Student's t Dist.)", linestyle="-", color="red", alpha=0.7)
plt.plot(vaR_combined.index, vaR_combined["VaR (FHS)"], label="VaR (FHS)", linestyle="-", color="green", alpha=0.7)

# Formatting
plt.title("Rolling 5% 2-Week VaR Estimates (Normal, t-Distribution, FHS)", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Value-at-Risk (%)", fontsize=12)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)

# Show the plot
plt.show()


In [ ]:
# Align returns with VaR data
returns = biweekly_returns.loc[var_norm_series.index]  
var_fhs_series = kospi_var_fhs_biweekly_df["VaR"].loc[var_norm_series.index]  # Use correct FHS VaR

# Run the Diebold-Mariano test
diebold_mariano_test("KOSPI (Biweekly)", returns, var_fhs_series, var_norm_series, alpha=0.05)


In [ ]:
# Ensure datasets have matching timestamps
returns = biweekly_returns.loc[var_norm_series.index]
var_fhs_series = kospi_var_fhs_biweekly_df["VaR"].loc[var_norm_series.index]

# Compute HITs for KOSPI
kospi_hits_norm_df = compute_and_export_hits("KOSPI (Biweekly)", returns, {"VaR_Normal": var_norm_series}, "kospi_hits_norm.csv")
kospi_hits_fhs_df = compute_and_export_hits("KOSPI (Biweekly)", returns, {"VaR_FHS": var_fhs_series}, "kospi_hits_fhs.csv")

# Plot HITs
plot_hits({"KOSPI Norm": kospi_hits_norm_df, "KOSPI FHS": kospi_hits_fhs_df}, "HITs for KOSPI - Normal vs. FHS VaR")

In [ ]:
# Apply likelihood ratio test for 2-week VaR HITs
likelihood_ratio_test("KOSPI", kospi_hits_norm_df)

In [ ]:
# Apply Christoffersen's Independence Test for 2-week VaR HITs
christoffersen_independence_test("KOSPI", kospi_hits_norm_df)

In [ ]:
# Load FHS VaR results (First Approach)
kospi_var_df = pd.read_csv("kospi_var_results.csv", parse_dates=["Date"], index_col="Date")

# Extract the 5% VaR (which corresponds to 95% confidence level in one-tailed tests)
kospi_var_5p_first = kospi_var_df["VaR 95%"] * np.sqrt(10)

# Load Gaussian VaR results (Second Approach)
var_norm_series = pd.read_csv("var_normal_series.csv", parse_dates=["Date"], index_col="Date")

In [ ]:
# Store standard GMZ results for 2-week GARCH VaR HITs
results_2_week_garch = {
    "KOSPI": ols_regression_with_wald_test("KOSPI", kospi_hits_norm_df, var_norm_series, var_col="0"),
}

# Print results for RiskMetrics
print(f"OLS Regression Summary for {"KOSPI"} (2-week GARCH VaR):")
print(results_2_week_garch["KOSPI"]["summary"])
print(f"Wald Test {"KOSPI"} (2-week GARCH VaR): {results_2_week_garch["KOSPI"]['stat']}\n")

In [ ]:
# Store results for RiskMetrics VaR HITs
results_garch_logit_probit = {
    "KOSPI": logit_probit_regression("KOSPI", kospi_hits_norm_df, var_norm_series, var_col="0"),
}

# Print results for RiskMetrics
print(f"Logit Regression Summary for {"KOSPI"} (2-week GARCH VaR):")
print(results_garch_logit_probit["KOSPI"]["logit_summary"])
print(f"Wald Test {"Logit"} (2-week GARCH VaR): {results_garch_logit_probit["KOSPI"]['logit_stat']}\n")
print(f"\nProbit Regression Summary for {"KOSPI"} (2-week GARCH VaR):")
print(results_garch_logit_probit["KOSPI"]["probit_summary"])
print(f"Wald Test {"Probit"} (2-week GARCH VaR): {results_garch_logit_probit["KOSPI"]['probit_stat']}\n")

# Q7 (b)

In [ ]:
# Load Student's t-distribution VaR results (Second Approach)
# var_t_series = pd.read_csv("var_t_series.csv", parse_dates=["Date"], index_col="Date")

# Find the common start date
common_start_date = max(kospi_var_5p_first.index.min(), var_norm_series.index.min())

# Trim both series to start from the common date
kospi_var_5p_first_aligned = kospi_var_5p_first.loc[common_start_date:]
var_norm_series = var_norm_series.loc[common_start_date:]

# Ensure alignment of both series' indices
kospi_var_5p_first_aligned = kospi_var_5p_first_aligned.reindex(var_norm_series.index, method="nearest")

# Plot the aligned VaR estimates
plt.figure(figsize=(12, 6))

# First Approach: Filtered Historical Simulation VaR
plt.plot(kospi_var_5p_first_aligned, label="FHS VaR (First Approach)", linestyle="-", color="blue", alpha=0.7)

# Second Approach: Student’s t-distribution VaR
plt.plot(var_norm_series, label="Normal VaR (Second Approach)", linestyle="-", color="red", alpha=0.7)

# Second Approach: Student’s t-distribution VaR
# plt.plot(var_t_series, label="Student's t VaR (Second Approach)", linestyle="-", color="red", alpha=0.7)

# Labels and title
# plt.title("Comparison of 5% VaR Estimates for KOSPI (FHS vs. Normal)", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Value-at-Risk (%)", fontsize=12)
plt.legend()
plt.grid(True, linestyle="-", alpha=0.5)

# Show plot
plt.show()

In [ ]:
# Find the common start date
common_start_date = max(kospi_var_5p_first.index.min(), var_norm_series.index.min())

# Trim both series to start from the common date
kospi_var_5p_first_aligned = kospi_var_5p_first.loc[common_start_date:]
var_t_series = var_norm_series.loc[common_start_date:]
returns = returns.loc[common_start_date:]

print(returns)
print(kospi_var_5p_first_aligned)
print(var_t_series)

var_t_series = var_t_series.squeeze()

# Run Diebold-Mariano test for VaR estimates
diebold_mariano_test("KOSPI", returns, kospi_var_5p_first_aligned, var_t_series, alpha=0.05)

In [ ]:
# Ensure datasets have matching timestamps
returns = biweekly_returns.loc[var_norm_series.index]
var_fhs_series = kospi_var_fhs_biweekly_df["VaR"].loc[var_norm_series.index]

# Compute HITs for KOSPI
kospi_hits_norm_df = compute_and_export_hits("KOSPI (Biweekly)", returns, {"VaR_Normal": var_norm_series['0']}, "kospi_hits_norm.csv")
kospi_hits_fhs_df = compute_and_export_hits("KOSPI (Biweekly)", returns, {"VaR_FHS": var_fhs_series}, "kospi_hits_fhs.csv")

# Plot HITs
plot_hits({"KOSPI Norm": kospi_hits_norm_df, "KOSPI FHS": kospi_hits_fhs_df}, "HITs for KOSPI - Normal vs. FHS VaR")